In [80]:
"""
All necessary imports
"""

import math
import csv

import numpy as np
import pandas as pd
import scipy.stats as stats
import scipy.spatial.distance as distance
import scipy

from pandas import read_csv
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

from minepy import MINE

import matplotlib.pyplot as plt
import seaborn
from mrmr import mrmr_classif
import pprint
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.feature_selection import RFECV
from sklearn.metrics import r2_score, mean_absolute_error
import pandas as pd
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import r2_score

from sklearn.feature_selection import SelectFromModel

from mendeleev import element
from mendeleev.fetch import fetch_table
import pandas as pd
import time
import ast
import re

In [182]:
converters = {
    'elem_in_site_order': ast.literal_eval,
    'elem_by_formula_order': ast.literal_eval,
    'num_electrons': ast.literal_eval,
    'eaffinity': ast.literal_eval,
    'dipole': ast.literal_eval,
    'covalent_rad': ast.literal_eval,
    'atomic_vol': ast.literal_eval,
    'atomic_rad': ast.literal_eval,
    'A1': ast.literal_eval,  # Converter for column 'A1'
    'A2': ast.literal_eval,  # Converter for column 'A2'
    'B1': ast.literal_eval,  # Converter for column 'B1'
    'B2': ast.literal_eval,  # Converter for column 'B2'
    'X': ast.literal_eval,   # Converter for column 'X'
}



df = pd.read_csv('Base_descriptors_sorted.csv', converters=converters)
#print(df_read)

In [183]:
pd.set_option('display.max_columns', None)
display(df.head(1))

,Unnamed: 0,formation_energy,elem_frac,elem_list,energy,formula,magnetic_moment,spin_index,num_p,num_d,num_f,atomic_rad,atomic_vol,covalent_rad,dipole,eaffinity,num_electrons,Unnamed: 17,elem_in_site_order,elem_by_formula_order,A1,A2,B1,B2,X
0,0,-1.15292,"[{Element Cr: 0.1}, {Element Co: 0.1}, {Elemen...","[Element Cr, Element Co, Element Ge, Element T...",-46.782036,Cr1Co1Ge1Te6P1,2.816,1,4,9,0,"[130.0, 125.0, 137.0, 160.0, 128.0]","[7.23, 6.7, 13.6, 20.5, 17.0]","[122.0, 111.00000000000001, 121.0, 136.0, 111....","[78.4, 57.71, 39.43, 37.0, 24.93]","[0.6659999999999999, 0.66225646, 1.232712, 1.9...","[24, 27, 32, 52, 15]",,"['Cr', 'Co', 'Ge', 'P', 'Te']","['Cr', 'Co', 'Ge', 'Te', 'P']","{'atomic_rad': 130.0, 'atomic_vol': 7.23, 'cov...","{'atomic_rad': 125.0, 'atomic_vol': 6.7, 'cova...","{'atomic_rad': 137.0, 'atomic_vol': 13.6, 'cov...","{'atomic_rad': 128.0, 'atomic_vol': 17.0, 'cov...","{'atomic_rad': 160.0, 'atomic_vol': 20.5, 'cov..."


In [175]:
'''for each element in elem site order:
    find the index in elem by formula order
    use this index to scrape each atom_rad till num electrons, then add them to a dict in that site

    for me: A1 will be atom radius, atom volume,
            A2 will be dipole polarizability, electron affinity
            A3 will be  covalent radius, num electrons

'''
# print(type(df['eaffinity']))
# print(type(df['eaffinity'][5]))
# print(df['eaffinity'][5][1])
descripts = ['atomic_rad', 'atomic_vol', 'covalent_rad', 'dipole', 'eaffinity',  ]
sites = ['A1', 'A2', 'B1', 'B2', 'X']
for(index, row) in df.iterrows():
    lms = row['elem_in_site_order']
    print(lms)
    for i in range(len(lms)):
        ndex = row['elem_by_formula_order'].index(lms[i])
        tmpdct = {}
        for d in descripts:
            tmpdct[d] = row[d][ndex]
    
        #row[sites[i]]=tmpdct
        df.at[index, sites[i]] = tmpdct
        


    

df.to_csv('Base_descriptors_sorted.csv', index=False)

['Cr', 'Co', 'Ge', 'P', 'Te']
['Cr', 'Co', 'Ge', 'Ge', 'Te']
['Cr', 'Co', 'Si', 'Ge', 'Te']
['Cr', 'Co', 'Si', 'P', 'Te']
['Cr', 'Co', 'Si', 'Si', 'Te']
['Cr', 'Co', 'P', 'P', 'Te']
['Cr', 'Cu', 'Ge', 'P', 'Te']
['Cr', 'Cu', 'Ge', 'Ge', 'Te']
['Cr', 'Cu', 'Si', 'Ge', 'Te']
['Cr', 'Cu', 'Si', 'P', 'Te']
['Cr', 'Cu', 'Si', 'Si', 'Te']
['Cr', 'Cu', 'P', 'P', 'Te']
['Cr', 'Fe', 'Ge', 'P', 'Te']
['Cr', 'Fe', 'Ge', 'Ge', 'Te']
['Cr', 'Fe', 'Si', 'Ge', 'Te']
['Cr', 'Fe', 'Si', 'P', 'Te']
['Cr', 'Fe', 'Si', 'Si', 'Te']
['Cr', 'Fe', 'P', 'P', 'Te']
['Cr', 'Ru', 'Ge', 'P', 'Te']
['Cr', 'Ru', 'Ge', 'Ge', 'Te']
['Cr', 'Ni', 'Ge', 'P', 'Te']
['Cr', 'Ni', 'Ge', 'Ge', 'Te']
['Cr', 'Ni', 'P', 'P', 'Te']
['Cr', 'Ru', 'Si', 'Ge', 'Te']
['Cr', 'Ni', 'Si', 'Ge', 'Te']
['Cr', 'Ni', 'Si', 'P', 'Te']
['Cr', 'Ru', 'Si', 'P', 'Te']
['Cr', 'Ni', 'Si', 'Si', 'Te']
['Cr', 'Ru', 'Si', 'Si', 'Te']
['Cr', 'Ru', 'P', 'P', 'Te']
['Cr', 'Cr', 'Ge', 'P', 'Te']
['Cr', 'Cr', 'Ge', 'Ge', 'Te']
['Cr', 'Cr', 'Si', 'Ge', 'Te'

In [181]:
print(type(df['A1'][3]))
df2 = pd.read_csv('organized_descriptors.csv', converters=converters)

print(type(df2['A1'][3]))

<class 'dict'>
<class 'dict'>
